In [42]:

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import keyboard
import pyperclip

import time
import os

import pandas as pd
import numpy as np

from access import user, passw





### ENTRAMOS CON SELENIUM A LA PÁGINA DE AIRDNA DE LA CIUDAD QUE NOS INTERESA, PASANDO POR EL LOGIN

# Especificamos nuestra web para scrapear
url_airdna = "https://app.airdna.co/data/login"

# Set-up del bot, incluyendo user-agent name change (*y uso de proxys)
service_airdna = Service(ChromeDriverManager().install())
chrome_options = Options()
chrome_options.add_argument('Mozilla/5.0.(Windows.NT.10.0;.Win64;.x64).AppleWebKit/537.36.(KHTML,.like.Gecko).Chrome/120.0.6099.216.Safari/537.36')
driver = webdriver.Chrome(service=service_airdna, options=chrome_options)
driver.maximize_window()
driver.get(url_airdna)

# Logging in 
time.sleep(round(np.random.uniform(1.5,4),2))
campo_email = driver.find_element(By.NAME, 'email')
campo_email.send_keys(user)
campo_pass = driver.find_element(By.NAME, 'password')
campo_pass.send_keys(passw)
button_login = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.TAG_NAME, 'button')))
button_login.click()

# Ahora entramos al endpoint que nos interesa directamente (url de STR Short Term Rentals en Málaga)
time.sleep(round(np.random.uniform(1.5,4),2))
url_str = 'https://app.airdna.co/data/es/30838?tab=active-str-listings' # Málaga 10.3k
# url_str = 'https://app.airdna.co/data/es/53223?tab=active-str-listings' # Alhaurín de la Torre 402
# url_str = 'https://app.airdna.co/data/es/53398?tab=active-str-listings' # Sayalonga 179
driver.get(url_str)







### ABRIMOS CHROME DEV TOOLS, EL PANEL NETWORK Y RECARGAMOS LA PAGINA 

keyboard.press_and_release('f12')
time.sleep(round(np.random.uniform(1.5,4),2))   # wait for DevTools to open
keyboard.press_and_release('ctrl+shift+p')
time.sleep(1)
keyboard.write('network')
time.sleep(round(np.random.uniform(1.5,4),2))   # writing network...
keyboard.press_and_release('Enter')
keyboard.press_and_release('ctrl+r')
time.sleep(round(np.random.uniform(10,12),2))   # wait for the page to reload





### SCORLLEAMOS HASTA EL ÚLTIMO ANUNCIO

xp1 = '//*[@id="__next"]/div/main/div/div[2]/div/div/div/div/div[2]/div[3]/div/div[2]/div/div/div/div[1]/div[1]/div/button'

scrolling_div = driver.find_element(By.XPATH, xp1)
scrolling_div.send_keys(Keys.PAGE_DOWN)
print('page down 1 ----------')
time.sleep(round(np.random.uniform(0.1,0.2),2))

xp1 = '//*[@id="__next"]/div/main/div/div[2]/div/div/div/div/div[2]/div[3]/div/div[2]/div/div/div/div[3]/div[1]/div/button'
scrolling_div = driver.find_element(By.XPATH, xp1)
scrolling_div.send_keys(Keys.PAGE_DOWN)
print('page down 2 ----------')
time.sleep(round(np.random.uniform(0.1,0.2),2))

xp1 = '//*[@id="__next"]/div/main/div/div[2]/div/div/div/div/div[2]/div[3]/div/div[2]/div/div/div/div[6]/div[1]/div/button'
scrolling_div = driver.find_element(By.XPATH, xp1)
scrolling_div.send_keys(Keys.PAGE_DOWN)
print('page down 3 ----------')
time.sleep(round(np.random.uniform(0.1,0.2),2))

error_notinteractable = 0
error_stale = 0
error_nosuch = 0
intento = 3
reviews = 50
while reviews > 49:
    try:
        intento += 1
        xp_root = '//*[@id="__next"]/div/main/div/div[2]/div/div/div/div/div[2]/div[3]/div/div[2]/div/div/div/'
        xp1 = xp_root + 'div[7]/div[1]/div/button'
        xp_reviews_div = xp_root + 'div[6]/div[2]/div/a/div/div[1]/div[2]/p'
        scrolling_div = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, xp1)))
        scrolling_div.send_keys(Keys.PAGE_DOWN)
        print('page down', intento, '----------')
        reviews_div = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, xp_reviews_div)))
        reviews = int(reviews_div.text.split(sep='(')[1].split(sep=')')[0])
        time.sleep(round(np.random.uniform(0.1,0.2),2))
    except StaleElementReferenceException:
        error_stale += 1
        print('STALE ELEMENT Error....!!!')
    except NoSuchElementException:
        error_nosuch += 1
        print('NO SUCH Error....!!!')
    except ElementNotInteractableException:
        error_notinteractable += 1
        print('ELEMENT NOT INTERACTABLE Error....!!!')

print('StaleElementReferenceException errors:', error_stale)
print('NoSuchElementException errors:', error_nosuch)
print('ElementNotInteractableException errors:', error_notinteractable)






### ACCEDEMOS A LA LISTA DE LOS JSONs GENERADOS

json_no = 0

tiempo_espera_CDT = round(np.random.uniform(1,1.5),2)

keyboard.write('/listings')
time.sleep(round(np.random.uniform(1.5,4),2))   # escribimos "/listings" como filtro de busqueda
for i in range(4):
    keyboard.press_and_release('tab')
    time.sleep(tiempo_espera_CDT)
keyboard.press_and_release('enter')             
for i in range(3):
    keyboard.press_and_release('down')
    time.sleep(tiempo_espera_CDT)
keyboard.press_and_release('Enter')             # seleccionamos Fetch/XHR como tipo de Response
for i in range(6):
    keyboard.press_and_release('tab')
    time.sleep(tiempo_espera_CDT)
keyboard.press_and_release('down')
time.sleep(tiempo_espera_CDT)





### ITERAMOS HASTA GUARDAR TODOS LOS JSONs

keyboard.press_and_release('enter')             # seleccionamos el primer recurso del filtro "/listings"
time.sleep(tiempo_espera_CDT)

keyboard.press_and_release('tab')
for i in range(3):
    keyboard.press_and_release('down')
    time.sleep(tiempo_espera_CDT)
keyboard.press_and_release('enter')             # seleccionamos el el tab Response de ese recurso
time.sleep(tiempo_espera_CDT)
keyboard.press_and_release('ctrl+a')            # seleccionamos todo el texto del Response
time.sleep(tiempo_espera_CDT)
keyboard.press_and_release('ctrl+c')            # copiamos el texto
time.sleep(tiempo_espera_CDT)

# guardamos el texto seleccionado en un JSON
json_no += 1
texto = pyperclip.paste()
folder_path = os.path.join(os.getcwd(), 'JSON LISTINGS')
file_path = os.path.join(folder_path, 'listing-'+str(json_no)+'.json')
with open(file_path, 'w') as file:
    file.write(texto)

copia_anterior = 'x'
copia_actual = 'y'

while True:

    keyboard.press_and_release('esc')               # salimos del Response
    time.sleep(tiempo_espera_CDT)
    keyboard.press_and_release('down')              # nos vamos al siguiente recurso del filtro "listing"
    time.sleep(tiempo_espera_CDT)

    keyboard.press_and_release('enter')             
    time.sleep(tiempo_espera_CDT)
    
    keyboard.press_and_release('tab')               # seleccionamos el tab Response de ese recurso
    time.sleep(tiempo_espera_CDT)
    keyboard.press_and_release('enter')             
    time.sleep(tiempo_espera_CDT)
    keyboard.press_and_release('ctrl+a')            # seleccionamos todo el texto del Response
    time.sleep(tiempo_espera_CDT)
    keyboard.press_and_release('ctrl+c')            # copiamos el texto
    time.sleep(tiempo_espera_CDT)
    # guardamos el texto seleccionado en un JSON
    json_no += 1
    texto = pyperclip.paste()
    copia_anterior = texto
    folder_path = os.path.join(os.getcwd(), 'JSON LISTINGS')
    file_path = os.path.join(folder_path, 'listing-'+str(json_no)+'.json')
    with open(file_path, 'w') as file:
        file.write(texto)

    keyboard.press_and_release('esc')               # salimos del Response
    time.sleep(tiempo_espera_CDT)
    keyboard.press_and_release('down')              # nos vamos al siguiente recurso del filtro "listing"
    time.sleep(tiempo_espera_CDT)

    keyboard.press_and_release('enter')             
    time.sleep(tiempo_espera_CDT)
    
    keyboard.press_and_release('tab')               # seleccionamos el el tab Response de ese recurso
    time.sleep(tiempo_espera_CDT)
    keyboard.press_and_release('enter')             
    time.sleep(tiempo_espera_CDT)
    keyboard.press_and_release('ctrl+a')            # seleccionamos todo el texto del Response
    time.sleep(tiempo_espera_CDT)
    keyboard.press_and_release('ctrl+c')            # copiamos el texto
    time.sleep(tiempo_espera_CDT)
    # guardamos el texto seleccionado en un JSON
    json_no += 1
    texto = pyperclip.paste()
    copia_actual = texto
    if copia_anterior == copia_actual:
        break
    folder_path = os.path.join(os.getcwd(), 'JSON LISTINGS')
    file_path = os.path.join(folder_path, 'listing-'+str(json_no)+'.json')
    with open(file_path, 'w') as file:
        file.write(texto)



page down 1 ----------
page down 2 ----------
page down 3 ----------
page down 4 ----------
page down 5 ----------
page down 6 ----------
page down 7 ----------
page down 8 ----------
page down 9 ----------
page down 10 ----------
page down 11 ----------
page down 12 ----------
page down 13 ----------
page down 14 ----------
page down 15 ----------
page down 16 ----------
page down 17 ----------
page down 18 ----------
page down 19 ----------
page down 20 ----------
page down 21 ----------
page down 22 ----------
page down 23 ----------
page down 24 ----------
page down 25 ----------
page down 26 ----------
page down 27 ----------
page down 28 ----------
page down 29 ----------
page down 30 ----------
page down 31 ----------
page down 32 ----------
STALE ELEMENT Error....!!!
page down 33 ----------
page down 34 ----------
page down 35 ----------
page down 36 ----------
page down 37 ----------
page down 38 ----------
page down 39 ----------
page down 40 ----------
page down 41 ---------